In [9]:
import pandas as pd
import os
from arch import arch_model
import pymc3 as pm
import numpy as np

In [6]:
df_fx = pd.read_csv(os.path.join('data', 'df_fx.csv'))

In [83]:
burnin = 3450

garch_dict = {}
gjr_garch_dict = {}
egarch_dict = {}
for col in df_fx[['AUD_USD', 'CHF_USD']].columns:
    ret = df_fx[col]
    
    fct1 = {}
    fct2 = {}
    fct3 = {}
    for step in range(df_fx.shape[0]-burnin-1):
        ret_loop = ret[0:burnin+step]
        garch = arch_model(ret_loop)
        res = garch.fit(update_freq=1, disp='off')
        forecasts = res.forecast(horizon=1)
        fct1[step] = forecasts.variance.dropna().iloc[0][0]

        gjr_garch = arch_model(ret_loop, p=1, o=1, q=1)
        res = gjr_garch.fit(update_freq=1, disp='off')
        forecasts = res.forecast(horizon=1)
        fct2[step] = forecasts.variance.dropna().iloc[0][0]

        egarch = arch_model(ret_loop, p=1, q=1, vol='EGARCH')
        res = egarch.fit(update_freq=1, disp='off')
        forecasts = res.forecast(horizon=1)
        fct3[step] = forecasts.variance.dropna().iloc[0][0]
    
    garch_dict[col] = fct1
    gjr_garch_dict[col] = fct2
    egarch_dict[col] = fct3

C:\Users\Daniel\Anaconda3\envs\dcuoliveira\lib\site-packages\arch\univariate\base.py:293: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.001512. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  data_scale_warning.format(orig_scale, rescale), DataScaleWarning
C:\Users\Daniel\Anaconda3\envs\dcuoliveira\lib\site-packages\arch\univariate\base.py:293: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.001512. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  data_scal

In [88]:
garch_df = pd.DataFrame.from_dict(garch_dict)
garch_df['tag'] = 'garch'
garch_df = garch_df.melt('tag')

gjr_garch_df = pd.DataFrame.from_dict(gjr_garch_dict)
gjr_garch_df['tag'] = 'gjr_garch'
gjr_garch_df = gjr_garch_df.melt('tag')

egarch_df = pd.DataFrame.from_dict(egarch_dict)
egarch_df['tag'] = 'egarch'
egarch_df = egarch_df.melt('tag')

forecast_all = pd.concat([garch_df, gjr_garch_df, egarch_df], axis=0)